In [ ]:
!pip install keras-self-attention


In [ ]:
# -*- coding: utf-8 -*-
"""ConvAi.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/14R83utTbKA_QSkZkWnLI7RtM5ht1huqE
"""

import pandas as pd

#import matplotlib.pyplot as plt 
import numpy as np
#from sklearn.preprocessing import StandardScaler
import os
from os.path import exists, join, basename, splitext

import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import *
from keras.layers import *
from keras import backend as K
import keras
from keras_self_attention import SeqSelfAttention,SeqWeightedAttention
from sklearn.metrics import mean_squared_error



np_input = np.load("./Input_Output/Input.npy")
np_output = np.load("./Input_Output/Output.npy")

print(np_input.shape)
print(np_output.shape)

class_output = []
for i in np_output:
  if i[0]==0:
    class_output.append([1,0,0,0])
  elif i[0]==0.33:
    class_output.append([0,1,0,0])
  elif i[0]==0.66:
    class_output.append([0,0,1,0])
  elif i[0]==1.:
    class_output.append([0,0,0,1])

class_output = np.array(class_output)
class_output = np.reshape(class_output,[121,4])

np_input = np_input[:120, :]
np_output = class_output[:120, :]

print(np_input.shape)
print(np_output.shape)


#np_input =  np.reshape(np_input, (-1,BATCH_SIZE, 120, 20))
#np_output =  np.reshape(np_output, (-1, BATCH_SIZE, 1))

model = Sequential()
model.add(Dense(16,input_shape=(120,20),activation='relu'))
model.add(Bidirectional(LSTM(16,return_sequences=True)))
model.add(Dropout(0.2))
model.add(SeqWeightedAttention())

model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='softmax'))


print(model.summary())

#plot_model(model, to_file='outputs.png')

model.compile(loss='mse', optimizer='adam',  metrics=['accuracy'])
history = model.fit(np_input, np_output, validation_split=0.2, epochs=40, batch_size=32)

print(history.history.keys())

model.save("Saved_Model")
epochs_range = range(40)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, history.history['loss'], label='Training Loss')
plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()



In [ ]:
!ls

sample_data


# 新段落

In [ ]:
test_gt = np.load("./Input_Output/Input_test.npy")
test_pred = np.load("./Input_Output/Output_test.npy")

y_test = model.predict(test_gt)
#print(y_test)

pred_list = []
for i in y_test:
  if tf.argmax(i).numpy()== 0:
    pred_list.append(0)
  elif tf.argmax(i).numpy()== 1:
    pred_list.append(0.33)
  elif tf.argmax(i).numpy()== 2:
    pred_list.append(0.66)
  elif tf.argmax(i).numpy()== 3:
    pred_list.append(1.)

pred_list = np.array(pred_list)
pred_list = np.reshape(pred_list,[-1,1])

np.save("./Output_data/Y_test",pred_list)

np.save("./Output_data/X_test", test_pred)

rms = mean_squared_error(test_pred,pred_list,squared=False)

x_test = np.load("./Output_data/X_test.npy")
y_test = np.load("./Output_data/Y_test.npy")

x_test = x_test.reshape((45,))
y_test = y_test.reshape((45,))
print(x_test)
print(y_test)

pcc_test = stats.pearsonr(x_test, y_test)
mse_test = mean_squared_error(x_test, y_test)

print("\nPearson coefficient for test set is : ")
print(pcc_test)
print("\nMean square error for test set is : ")
print(mse_test)
print("")

[0.33 0.   0.66 0.66 1.   0.66 0.33 0.66 0.66 0.66 0.33 0.33 0.66 0.66
 0.33 1.   1.   1.   1.   0.66 1.   0.66 1.   1.   0.33 0.   0.66 0.33
 1.   1.   0.   0.66 0.33 0.66 0.33 0.66 1.   1.   0.66 0.66 1.   0.66
 0.   0.33 0.66]
[0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66
 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66
 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66
 0.66 0.66 0.66]

Pearson coefficient for test set is : 
(nan, nan)

Mean square error for test set is : 
0.09631555555555557



/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
